In [ ]:
!pip install accelerate bitsandbytes peft transformers trl==0.12.0 datasets deepspeed
!pip install torch --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.16.1-py3-none-any.whl size

In [ ]:
import logging
import sys
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, TrainingArguments)
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Logging configuration
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# Model & Training Config
training_config = {
    "bf16": True,
    "learning_rate": 5.0e-5,
    "num_train_epochs": 1,
    "output_dir": "/content/drive/MyDrive/fine_tuned_model",
    "per_device_train_batch_size": 2,  # Reduced for Colab
    "per_device_eval_batch_size": 2,
    "save_steps": 100,
    "save_total_limit": 1,
    "logging_steps": 60,
    "gradient_checkpointing": True,
    "gradient_accumulation_steps": 4,  # Helps with memory efficiency
    "warmup_ratio": 0.1,
}

peft_config = LoraConfig(
    r=8,  # Smaller dimension for Lora layers
    lora_alpha=32,
    target_modules=["qkv_proj", "o_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

train_args = TrainingArguments(**training_config)

In [ ]:
checkpoint = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
# Load opus-100 dataset
dataset = load_dataset("Helsinki-NLP/opus-100", "en-id")

# Take 100,000 samples for training and all validation data
train_data = dataset["train"].shuffle(seed=42).select(range(20000))
valid_data = dataset["validation"]

print(train_data[0])

README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/132k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': '♪ Ch-ch-ch-ch-changes', 'id': '♪ Ch-ch-ch-ch-changes ♪'}}


In [ ]:
# Preprocess function
def preprocess_function(example):
    input_text = example['translation']['en']
    target_text = example['translation']['id']

    # Input prompt with target text
    formatted_text = (
        f"Translate the following text from English to Indonesian:\n\n"
        f"English: {input_text}\nIndonesian: {target_text}"
    )

    return {"text": formatted_text}

In [ ]:
# Preprocess and map datasets
train_dataset = train_data.map(preprocess_function, remove_columns=["translation"])
valid_dataset = valid_data.map(preprocess_function, remove_columns=["translation"])

# Debug: Verify a few examples
print("Example from Training Dataset:")
print(train_dataset[1])

Example from Training Dataset:
{'text': 'Translate the following text from English to Indonesian:\n\nEnglish: I know how to use this thing!\nIndonesian: Aku tahu cara menggunakannya !'}


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    dataset_text_field="text",
    packing=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

[2024-12-18 01:41:45,245] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
60,1.725500


TrainOutput(global_step=112, training_loss=1.5384480612618583, metrics={'train_runtime': 9934.1867, 'train_samples_per_second': 0.091, 'train_steps_per_second': 0.011, 'total_flos': 2.0518862390820864e+16, 'train_loss': 1.5384480612618583, 'epoch': 0.9933481152993349})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/fine_tuned_model/model_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer")

('/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer/tokenizer.model',
 '/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_model/phi3_tokenizer/tokenizer.json')

In [ ]:
# Save trained model
trainer.save_model("/content/drive/MyDrive/fine_tuned_model/phi3-finetuned")

In [ ]:
# Evaluate
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 1.4134738445281982, 'eval_runtime': 328.8858, 'eval_samples_per_second': 0.295, 'eval_steps_per_second': 0.149, 'epoch': 0.9933481152993349}


In [ ]:
def generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence):
    """
    Generate a few-shot prompt for English to Indonesian translation.

    Parameters:
        english_sentences (list): List of example English sentences.
        indonesian_targets (list): List of corresponding Indonesian translations.
        new_sentence (str): The new English sentence to translate.

    Returns:
        str: Formatted few-shot prompt.
    """
    # Validate inputs
    if len(english_sentences) != len(indonesian_targets):
        raise ValueError("The number of English sentences and Indonesian targets must match.")

    # Start with "Examples"
    examples = ""
    for eng, indo in zip(english_sentences, indonesian_targets):
        examples += f"English: {eng}\nIndonesian: {indo}\n\n"

    # Add the new sentence for translation
    prompt = (
        examples +
        "Translate the following English sentences to Indonesian:\n"
        + f"\nEnglish: {new_sentence}\nIndonesian:"
    )

    return prompt

In [ ]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu disudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."



# Training Data Examples
english_sentences = [
    "I feel sorry for her.",
    # "I am always with you.",
    # "We didn't understand much of what he said, only that he was suffering.",
    # "Hey, good morning!",
    # "That's never happened.",
    # "It's just, the way I asked you, I kind of left out the most important part."
]

indonesian_targets = [
    "Aku merasa kasihan padanya.",
    # "Saya akan selalu bersamamu.",
    # "Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.",
    # "Hei, selamat pagi!",
    # "Itu tidak pernah terjadi.",
    # "Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya."
]

# The new sentence to translate
new_sentence = "Yeah, we've been teaching him, but really, he's been teaching us."

# Generate the prompt
prompt = generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence)
print(prompt)

In [ ]:
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)

output = model.generate(
      input_ids=inputs,
      max_new_tokens=60,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(output[0], skip_special_tokens=True)

if len(output) == 0:
    print("Warning: Model output is empty!")
else:
    print(response)

English: I feel sorry for her.
Indonesian: Aku merasa kasihan padanya.

Translate the following English sentences to Indonesian:

English: Yeah, we've been teaching him, but really, he's been teaching us.
Indonesian: Ya, kita telah mengajar dia, tapi sebenarnya dia telah mengajar kita.


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure the nltk 'punkt' tokenizer is downloaded
nltk.download('punkt_tab')

def calculate_bleu_single_reference(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score for a single predicted translation against one reference.

    Parameters:
        reference_sentence (str): The correct reference translation.
        candidate_sentence (str): The predicted translation.

    Returns:
        float: The BLEU score.
    """
    # Tokenize the reference sentence and candidate sentence
    tokenized_reference = [nltk.word_tokenize(reference_sentence)]
    tokenized_candidate = nltk.word_tokenize(candidate_sentence)

    # Use smoothing to handle short sentences
    smoothing_function = SmoothingFunction().method1

    # Calculate BLEU score
    bleu_score = sentence_bleu(tokenized_reference, tokenized_candidate, smoothing_function=smoothing_function)
    return bleu_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu di sudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# Single reference translation
reference_sentence = "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
candidate_sentence = "Ya, kita telah mengajar dia, tapi sebenarnya dia telah mengajar kita."

# Calculate BLEU score
bleu_score = calculate_bleu_single_reference(reference_sentence, candidate_sentence)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.1030


In [ ]:
import pandas as pd

In [ ]:
def extract_translation(output):
    """
    Extract the translation from the model's output after the second 'Indonesian:'.

    Parameters:
        output (str): The raw output from the model.

    Returns:
        str: The extracted translation.
    """
    # Find the second occurrence of 'Indonesian:'
    parts = output.split("Indonesian:")
    if len(parts) > 2:
        translation = parts[2].strip()  # Extract the part after the second 'Indonesian:'
    else:
        translation = parts[-1].strip()  # Fallback to the last part if 'Indonesian:' appears once

    return translation

In [ ]:
print(extract_translation(""))

In [ ]:
def evaluate_model(model, tokenizer, test_data, output_path, english_sentences, indonesian_targets):
    results = []

    for example in test_data:
        # Extract input and target from the dataset
        english_sentence = example["translation"]["en"]
        reference_sentence = example["translation"]["id"]

        # Generate the prompt
        prompt = generate_few_shot_prompt(english_sentences, indonesian_targets, english_sentence)

        # Tokenize and generate prediction
        inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
        output = model.generate(
          input_ids=inputs,
          max_new_tokens=60,
          num_return_sequences=1,
          eos_token_id=tokenizer.eos_token_id,
        )
        raw_output = tokenizer.decode(output[0], skip_special_tokens=True)
        prediction_sentence = extract_translation(raw_output)

        # Calculate BLEU score
        bleu_score = calculate_bleu_single_reference(reference_sentence, prediction_sentence)

        # Store results
        results.append({
            "Input (English)": english_sentence,
            "Reference (Indonesian)": reference_sentence,
            "Prediction (Indonesian)": prediction_sentence,
            "BLEU Score": bleu_score,
        })

    # Save results to Excel
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")

    return results_df

In [ ]:
# Load test data
test_data = dataset["test"].shuffle(seed=42).select(range(100))

test_data[0]

{'translation': {'en': "Pat's under a lot of pressure.",
  'id': 'Pat berada dalam banyak tekanan.'}}

In [ ]:
# Path
one_shot_path = "/content/drive/MyDrive/fine_tuned_model/one_shot_results.xlsx"

# Evaluate the model on the test data and save results
results_df_one_shot = evaluate_model(model, tokenizer, test_data, one_shot_path, english_sentences, indonesian_targets)

results_df_one_shot.head()

Results saved to /content/drive/MyDrive/fine_tuned_model/one_shot_results.xlsx


,Input (English),Reference (Indonesian),Prediction (Indonesian),BLEU Score
0,Pat's under a lot of pressure.,Pat berada dalam banyak tekanan.,English: I feel sorry for her.\nIndonesian: Ak...,0.023952
1,Why did You show yourself to only me?,Mengapa Kau menampakkan diri hanya padaku?,English: I feel sorry for her.\nIndonesian: Ak...,0.005919
2,"Gunner, last chance.","Gunner, kesempatan terakhir.",English: I feel sorry for her.\nIndonesian: Ak...,0.012070
3,- It's safe enough.,- Ini aman.,English: I feel sorry for her.\nIndonesian: Ak...,0.011408
4,This information is only available to Last.fm ...,Informasi ini hanya tersedia bagi pengguna Las...,English: I feel sorry for her.\nIndonesian: Ak...,0.089549


In [ ]:
few_shot_path = "/content/drive/MyDrive/fine_tuned_model/few_shot_results.xlsx"

# Training Data Examples
english_sentences = [
    "I feel sorry for her.",
    "I am always with you.",
    "We didn't understand much of what he said, only that he was suffering.",
    "Hey, good morning!",
    "That's never happened.",
    "It's just, the way I asked you, I kind of left out the most important part."
]

indonesian_targets = [
    "Aku merasa kasihan padanya.",
    "Saya akan selalu bersamamu.",
    "Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.",
    "Hei, selamat pagi!",
    "Itu tidak pernah terjadi.",
    "Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya."
]

results_df_few_shot = evaluate_model(model, tokenizer, test_data, few_shot_path, english_sentences, indonesian_targets)

results_df_few_shot.head()